In [36]:
## writing dsq files
corrupt_pi <- c(1:100)/100
writefunc <-"  "
for(i in seq(length(corrupt_pi))){
 writefunc <- rbind(writefunc, paste0('module load R/3.6.1; cd /gpfs/ysm/pi/zhao-data/wd262/sc_immune/sc_immune; Rscript --vanilla ./Simulation_Results/get_simu_rst.v2.R 5 500 8 ',corrupt_pi[i],' 1 1 100 ',100+i," corrupt"))
}

write.table(writefunc, file ='/gpfs/loomis/scratch60/zhao/wd262/sc_immune/simulation/corrupt_dsq_jobs.sh', row.names = FALSE, col.names = FALSE, quote=FALSE)

In [37]:
## writing dsq files
tissue_number <- c(3:40)
writefunc <-"  "
for(i in seq(length(tissue_number))){
 writefunc <- rbind(writefunc, paste0('module load R/3.6.1; cd /gpfs/ysm/pi/zhao-data/wd262/sc_immune/sc_immune; Rscript --vanilla ./Simulation_Results/get_simu_rst.v2.R 5 500 ', tissue_number[i],' 0.6 1 1 100 ',100+i," tissue_number"))
}

write.table(writefunc, file ='/gpfs/loomis/scratch60/zhao/wd262/sc_immune/simulation/tissue_number_dsq_jobs.sh', row.names = FALSE, col.names = FALSE, quote=FALSE)

In [38]:
gene_number <- c(5:200)*10
writefunc <-"  "
for(i in seq(length(gene_number))){
 writefunc <- rbind(writefunc, paste0('module load R/3.6.1; cd /gpfs/ysm/pi/zhao-data/wd262/sc_immune/sc_immune; Rscript --vanilla ./Simulation_Results/get_simu_rst.v2.R 5 ',gene_number[i],' 8 0.6 1 1 100 ',100+i," gene_number"))
}

write.table(writefunc, file ='/gpfs/loomis/scratch60/zhao/wd262/sc_immune/simulation/gene_number_dsq_jobs.sh', row.names = FALSE, col.names = FALSE, quote=FALSE)

In [ ]:
## writing dsq files
corrupt_pi <- c(1:100)/100
writefunc <-"  "
for(i in seq(length(corrupt_pi))){
 writefunc <- rbind(writefunc, paste0('module load R/3.6.1; cd /gpfs/ysm/pi/zhao-data/wd262/sc_immune/sc_immune; Rscript --vanilla ./Simulation_Results/get_simu_rst.v2.R 5 500 8 ',corrupt_pi[i],' 1 1 100 ',100+i," corrupt"))
}

write.table(writefunc, file ='/gpfs/loomis/scratch60/zhao/wd262/sc_immune/simulation/corrupt_dsq_jobs.sh', row.names = FALSE, col.names = FALSE, quote=FALSE)

In [29]:
source("benchmarking/benchmarking.R")
source('Batch_Correction/platform_batch_correction_combat.R')

In [ ]:
fraction_est <- function(Y0, SAME_rst, 
                        target_tissue, celltype_used_list,Celltype_used = "Celltype_used",
                        method = c("tranSig","music","w_empirical","cibersortx"), adj.to.sc = TRUE
){
  YSG <- SAME_rst$YSG
  vg <- SAME_rst$vg
  rownames(vg) <- YSG
  X <- SAME_rst$X[[target_tissue]]
  N <- ncol(Y0)
  if(adj.to.sc){
    rst_bulk <- Y_batch_correct(Y0, X, YSG)
    YSG <- rst_bulk$YSG
    Y0_adj <- rst_bulk$bulk_to_sc
    Y0_adj <- Y0_adj[YSG,] #TPM space
  }else{
    YSG <- intersect(YSG, rownames(Y0))
    Y0_adj <- Y0[YSG,]
  }
    
  rst <- list()  
  if("tranSig" %in% method){
    z_est_tranSig <- t(sapply(c(1:N), function(n) nnls(vg[YSG,],(Y0_sc[YSG,n]))$x))
    colnames(z_est_tranSig) <- celltype_used_list
    rst$z_est_tranSig <- z_est_tranSig                          
  }
  if("music" %in% method){
    z_est_music <- run_music(X,Y0,method="music")
    colnames(z_est_music) <- celltype_used_list  
    rst$z_est_music <- z_est_music   
  }
  if("w_empirical" %in% method){
    z_est_empirical <- run_music(X,Y0,method="nnls")
    colnames(z_empirical) <- celltype_used_list  
    rst$z_est_empirical <- z_est_empirical  
  }
  if("cibersortx" %in% method){
      
  }
  
  
                         
  rst$YSG <- YSG
  return(rst)
}

In [20]:
output_directory <- paste0("/gpfs/loomis/scratch60/zhao/wd262/sc_immune/simulation/corrupt/",60)
X_target <- readRDS(paste0(output_directory,"/X_1.rds"))
Y0 <- read.table(paste0(output_directory, "/Y0.txt"), sep = " ")